In [ ]:
# ============================================================
# CSV (5 file) -> predict stress_level -> emoji -> insert ke MySQL
#
# Output table columns:
# stress_level_id (AUTO INC), user_id, date, stress_level, gpa,
# extracurricular_hour_per_day, physical_activity_hour_per_day,
# sleep_hour_per_day, study_hour_per_day, social_hour_per_day,
# emoji, is_restored, created_at
# ============================================================

import os
import numpy as np
import pandas as pd
from pathlib import Path
import joblib
from sqlalchemy import create_engine, text

# =========================
# 0) INPUT FILES
# =========================
CSV_FILES = [
    ("Akbar", Path("./Our Data - Akbar.csv"), 1),
    ("Kaleb", Path("./Our Data - Kaleb.csv"), 2),
    ("Epin",  Path("./Our Data - Epin.csv"),  3),
    ("Adel",  Path("./Our Data - Adel.csv"),  4),
    ("Sye",   Path("./Our Data - Sye.csv"),   5),
]

# Range yang dipakai
START_DATE = pd.Timestamp("2025-11-21")
END_DATE   = pd.Timestamp("2026-01-22")

# Model
ARTIFACT_PATH = Path("current_stress_pipeline.joblib")

# =========================
# 1) DB CONFIG (ENV recommended)
# =========================
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", "3306"))
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
    pool_pre_ping=True
)

# =========================
# 2) Helpers
# =========================
month_map = {
    "January":1,"February":2,"March":3,"April":4,"May":5,"June":6,"July":7,"August":8,"September":9,"October":10,"November":11,"December":12,
    "Januari":1,"Februari":2,"Maret":3,"April":4,"Mei":5,"Juni":6,"Juli":7,"Agustus":8,"September":9,"Oktober":10,"November":11,"Desember":12
}

def to_num(x):
    """Handle string angka dengan koma (3,82 / 8,5) -> float"""
    if pd.isna(x):
        return np.nan
    if isinstance(x, str):
        x = x.strip().replace(",", ".")
        x = x.replace(" ", "")
    return pd.to_numeric(x, errors="coerce")

def detect_ymd_cols(df: pd.DataFrame):
    """Detect kolom tahun/bulan/tanggal (kadang jadi Unnamed:0/1/2)"""
    if {"Tahun","Bulan","Tanggal"}.issubset(df.columns):
        return "Tahun", "Bulan", "Tanggal"
    # fallback: 3 kolom pertama (Kaleb/Epin/Adel/Sye csv kamu)
    return df.columns[0], df.columns[1], df.columns[2]

def normalize_one_csv(csv_path: Path, user_id: int) -> pd.DataFrame:
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV not found: {csv_path}")

    df = pd.read_csv(csv_path)

    ycol, mcol, dcol = detect_ymd_cols(df)

    year = pd.to_numeric(df[ycol], errors="coerce").astype("Int64")
    month = df[mcol].astype(str).str.strip()
    day = pd.to_numeric(df[dcol], errors="coerce").astype("Int64")
    month_num = month.map(month_map)

    date = pd.to_datetime(dict(year=year, month=month_num, day=day), errors="coerce")

    # Kolom data sesuai CSV kamu
    col_study = "Study Hours Per Day (Normalnya Weekday 6)"
    col_extra = "Extracurricular Hours Per Day"
    col_sleep = "Sleep Hours Per Day"
    col_social = "Social Hours Per Day (Normalnya 2, kerkom/belajar bareng dihitung juga)"
    col_phys = "Physical Activity Hours Per Day (Normalnya 0.5)"
    col_gpa = "GPA"

    out = pd.DataFrame({
        "user_id": user_id,
        "date": date,
        "study_hour_per_day": df.get(col_study),
        "extracurricular_hour_per_day": df.get(col_extra),
        "sleep_hour_per_day": df.get(col_sleep),
        "social_hour_per_day": df.get(col_social),
        "physical_activity_hour_per_day": df.get(col_phys),
        "gpa": df.get(col_gpa),
    })

    # convert numbers properly (koma -> titik)
    for c in [
        "study_hour_per_day",
        "extracurricular_hour_per_day",
        "sleep_hour_per_day",
        "social_hour_per_day",
        "physical_activity_hour_per_day",
        "gpa"
    ]:
        out[c] = out[c].apply(to_num)

    # filter range
    out = out[out["date"].between(START_DATE, END_DATE)].copy()
    out = out.sort_values("date").reset_index(drop=True)

    return out

# =========================
# 3) Load all CSVs
# =========================
all_df = []
for name, path, uid in CSV_FILES:
    tmp = normalize_one_csv(path, uid)
    print(f"{name}: rows_in_range={len(tmp)}")
    all_df.append(tmp)

df = pd.concat(all_df, ignore_index=True)
df = df.sort_values(["user_id","date"]).reset_index(drop=True)

print("\nTotal rows:", len(df))
print("Rows per user:", df.groupby("user_id").size().to_dict())

# =========================
# 4) Validation (harus tidak ada NA di kolom wajib)
# =========================
required = [
    "gpa","extracurricular_hour_per_day","physical_activity_hour_per_day",
    "sleep_hour_per_day","study_hour_per_day","social_hour_per_day"
]
bad = df[df[required].isna().any(axis=1)]
if not bad.empty:
    print("\n❌ Masih ada nilai kosong/invalid di kolom wajib. Fix CSV dulu:")
    print(bad[["user_id","date"] + required].to_string(index=False))
    raise SystemExit("Stop: Fix CSV lalu rerun.")

# =========================
# 5) created_at random 19:00–23:00, is_restored=0
# =========================
rng_time = np.random.default_rng(26)

def random_created_at(d: pd.Timestamp) -> str:
    h = int(rng_time.integers(19, 24))
    m = int(rng_time.integers(0, 60))
    s = int(rng_time.integers(0, 60))
    return f"{d.date()} {h:02d}:{m:02d}:{s:02d}"

df["created_at"] = df["date"].apply(random_created_at)
df["is_restored"] = 0

# =========================
# 6) Load model & predict stress_level
# =========================
if not ARTIFACT_PATH.exists():
    raise FileNotFoundError(f"Model artifact not found: {ARTIFACT_PATH}")

artifact = joblib.load(ARTIFACT_PATH)
pipeline = artifact.get("pipeline")
feature_names = list(artifact.get("feature_names", []))
if pipeline is None or not feature_names:
    raise ValueError("Artifact invalid. Need keys: pipeline, feature_names")

rename_map = {
    "gpa": "GPA",
    "study_hour_per_day": "Study_Hours_Per_Day",
    "sleep_hour_per_day": "Sleep_Hours_Per_Day",
    "social_hour_per_day": "Social_Hours_Per_Day",
    "physical_activity_hour_per_day": "Physical_Activity_Hours_Per_Day",
    "extracurricular_hour_per_day": "Extracurricular_Hours_Per_Day",
}
feat_df = df.rename(columns=rename_map).copy()

# optional: Academic performance columns (kalau model butuh)
def categorize_academic_performance(gpa: float) -> str:
    if gpa >= 3.5: return "Excellent"
    if 3.0 <= gpa < 3.5: return "Good"
    if 2.0 <= gpa < 3.0: return "Fair"
    return "Poor"

mapping_perf = {"Poor": 0, "Fair": 1, "Good": 2, "Excellent": 3}

if "Academic_Performance" in feature_names and "Academic_Performance" not in feat_df.columns:
    feat_df["Academic_Performance"] = feat_df["GPA"].astype(float).apply(categorize_academic_performance)

if "Academic_Performance_Encoded" in feature_names and "Academic_Performance_Encoded" not in feat_df.columns:
    perf = feat_df["GPA"].astype(float).apply(categorize_academic_performance)
    feat_df["Academic_Performance_Encoded"] = perf.map(mapping_perf).astype(int)

missing_feats = [c for c in feature_names if c not in feat_df.columns]
if missing_feats:
    raise KeyError(f"Missing model features: {missing_feats}\nAvailable: {list(feat_df.columns)}")

X = feat_df[feature_names].copy()
pred = pipeline.predict(X).astype(int)
df["stress_level"] = pred

# =========================
# 7) Emoji rule
# stress=0 -> emoji 0-1
# stress=1 -> emoji 2
# stress=2 -> emoji 3-4
# =========================
rng_emoji = np.random.default_rng(26)
emoji = np.empty_like(pred)
for i, s in enumerate(pred):
    if s == 0:
        emoji[i] = int(rng_emoji.integers(0, 2))
    elif s == 1:
        emoji[i] = 2
    else:
        emoji[i] = int(rng_emoji.integers(3, 5))
df["emoji"] = emoji

# =========================
# 8) TRUNCATE + INSERT to DB
# =========================
with engine.begin() as conn:
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0;"))
    conn.execute(text("TRUNCATE TABLE stress_levels;"))
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1;"))

insert_sql = text("""
INSERT INTO stress_levels (
  user_id, date, stress_level, gpa,
  extracurricular_hour_per_day,
  physical_activity_hour_per_day,
  sleep_hour_per_day,
  study_hour_per_day,
  social_hour_per_day,
  emoji, is_restored, created_at
) VALUES (
  :user_id, :date, :stress_level, :gpa,
  :extracurricular_hour_per_day,
  :physical_activity_hour_per_day,
  :sleep_hour_per_day,
  :study_hour_per_day,
  :social_hour_per_day,
  :emoji, :is_restored, :created_at
)
""")

payload = df.assign(date=df["date"].dt.date.astype(str)).to_dict(orient="records")
with engine.begin() as conn:
    conn.execute(insert_sql, payload)

print("\n✅ DONE. Inserted rows:", len(payload))
print("Stress dist:", pd.Series(pred).value_counts().sort_index().to_dict())
print("Emoji dist :", pd.Series(emoji).value_counts().sort_index().to_dict())
print(df[["user_id","date","stress_level","emoji","created_at"]].head(15).to_string(index=False))
